In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Load the Train set
train_set = pd.read_csv("../input/titanic/train.csv")
test_set=pd.read_csv("../input/titanic/test.csv")
train_set.tail(10)

In [ ]:
# inspect the structure etc.
print(train_set.info(), "\n")
print(train_set.shape)

In [ ]:
train_set['Survived'].value_counts()

In [ ]:
sns.countplot(train_set['Survived'])
plt.show()
print('Percent of fraud transaction: ',len(train_set[train_set['Survived']==1])/len(train_set['Survived'])*100,"%")
print('Percent of normal transaction: ',len(train_set[train_set['Survived']==0])/len(train_set['Survived'])*100,"%")

In [ ]:
# missing values in Train set df
train_set.isnull().sum()

In [ ]:
round(100*(test_set.isnull().sum().sort_values(ascending=False)/len(test_set.index)), 2)

##### As we can see above Cabin and Age have maximum chunk of missing values. So we need to check or get inference for them to imput with some values

In [ ]:
train_set.Age.describe()

Lets first use Name colums to drive the new Title columns which can be helpful in idea of age by grouping them

In [ ]:
train_set['Title']=train_set['Name'].map(lambda x: x.split(',')[1].split('.')[0].lstrip())
test_set['Title']=test_set['Name'].map(lambda x: x.split(',')[1].split('.')[0].lstrip())
train_set.head()

In [ ]:
train_set['Title'].value_counts()

###### we can see there are unnecessary catogories for Dr, Major and and some title for male as sir and Don and etc. Lets keep four categories only Mr., Miss. , Mrs, train_set

In [ ]:
print(train_set.info())

In [ ]:
#Check the list of values in title column
train_set.Title.unique()

In [ ]:
# lets sort the remaining other categories in title to various sub category of Mr, Miss, mrs, train_set
title_list=['Mrs', 'Mr', 'Master', 'Miss']
train_set.loc[~train_set['Title'].isin(title_list),['Age','Sex','Title']]

From the above we can categorize as below
1. Don, major, Capt,Jonkheer, Rev, Col as MR as Sex=Male and age is also greater that 30.
2. Countess, MMe can be categorized as Mrs as sex = Female and age is greater than 30
3. Mlle, Ms as Miss because sex=Female, age<30
4. Dr will be categorized into Mr and Mrs on the basis of Sex


In [ ]:
# function to bucket other titles into major 4
def fix_title(x):
    title=x['Title']
    if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col','Sir']:
        return 'Mr'
    elif title in ['the Countess', 'Mme','Lady','Dona']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='Male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [ ]:
train_set['Title']=train_set.apply(fix_title, axis=1)
train_set['Title'].value_counts()

In [ ]:
test_set['Title']=test_set.apply(fix_title, axis=1)
test_set['Title'].value_counts()

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4}
train_set['Title'] = train_set['Title'].map(title_mapping)
train_set['Title'] = train_set['Title'].fillna(0)

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4}
test_set['Title'] = test_set['Title'].map(title_mapping)
test_set['Title'] = test_set['Title'].fillna(0)

In [ ]:
train_set.Age.isnull().sum()

### **Missing Value treatment**

In [ ]:
#Check mean  on title Subclass w.r.t Age
train_set.groupby(['Title'])['Age'].describe()

This excerice of checking mean, median into subclass is important as it will help in decision on imputing Null values on Age as title give relative information on Age

In [ ]:
train_set.groupby(['Title'])['Age'].median()

1. Master Class have Age mean 4.57 and median 3.5
2. Miss Class have Age mean and median approx 21.0
3. Mr. Class have Age mean 32 and median 30
4. Mrs class have Age mean 36 and median 35. 

So Mean and Median are very close by on all the values so lets impute the missing values in Age with Median

In [ ]:
round(100*(train_set.isnull().sum().sort_values(ascending=False)/len(train_set.index)), 2)

In [ ]:
# Total Nullvalues in Age Column
train_set.Age.isnull().sum()

In [ ]:
title = train_set['Title'].value_counts()
a= dict(title)
for keys, values in a.items():
    print('Value of {} Class with Null Values {other}'.format(keys, other=train_set.loc[(train_set.Title==keys),['Age']].isnull().sum()))

In [ ]:
#Impute Missing values in Age Column
for keys, values in a.items():
    missing_val=train_set.loc[(train_set.Title==keys) & ~(train_set.Age.isnull()),['Age']].median(axis=0, skipna=True).astype('float')
    train_set.loc[(train_set.Title==keys) & (train_set.Age.isnull()),'Age']=train_set.loc[(train_set.Title==keys) & (train_set.Age.isnull()),'Age'].replace(np.nan,missing_val.median())

In [ ]:
title = train_set['Title'].value_counts()
b= dict(title)
for keys, values in a.items():
    print('Value of {} Class with Null Values {other}'.format(keys, other=test_set.loc[(test_set.Title==keys),['Age']].isnull().sum()))

In [ ]:
#Impute Missing values in Age Column
for keys, values in b.items():
    missing_val=test_set.loc[(test_set.Title==keys) & ~(test_set.Age.isnull()),['Age']].median(axis=0, skipna=True).astype('float')
    test_set.loc[(test_set.Title==keys) & (test_set.Age.isnull()),'Age']=test_set.loc[(test_set.Title==keys) & (test_set.Age.isnull()),'Age'].replace(np.nan,missing_val.median())

In [ ]:
# After Imputation on Age Colums verify the null values
train_set.Age.isnull().sum()

In [ ]:
test_set.Age.isnull().sum()

In [ ]:
test_set['Fare'].fillna(test_set['Fare'].median(), inplace=True)

Now Only Cabin have massive chunk 
and Very few in Embarked have the missing values. We need to find pattern in Cabin and for Embarked we can remove and impute from Median

In [ ]:
# GEt the unique set of Value of Cabin
train_set.Cabin.unique()

In [ ]:
# Lets see the cabin with passenger class
class_cabin=train_set.groupby(['Pclass'])['Cabin'].count()
class_cabin

In [ ]:
# No Pclass
train_set.Pclass.value_counts()

In [ ]:
cls = train_set['Pclass'].value_counts()
for key, value in (dict(cls)).items():
    print('Value of {} passenger Class with Null Values {other}'.format(key,other=train_set.loc[(train_set.Pclass==key),['Cabin']].isnull().sum()))

In [ ]:
train_set.loc[(train_set.Pclass==1) & ~(train_set.Cabin.isnull()),['Cabin']]


 We Can see the Cabin list is not giving much information and every Class of Passanger have some missing values. We can see from the above few Steps that PClass 3 is lower class have highest missing values.
 1. 1st Class have most popluated have given the Cabin and can say the First letter is the kind of Deck# as we have in Ships for ex A, C, B, T, D, E
 2. Same as in rest of the class we can relate that if there is Cabin List no is there so particulary assigned then is attached to some deck and for other cases where cabin # is Null we can say it as General/ unkwnown. Let say it GNR(general).

In [ ]:
# Lets have Deck # as separte Columns and null as GNR
train_set['Deck']=pd.Series(train_set.loc[~(train_set.Cabin.isnull()),['Cabin']].values.flatten()).astype('str').str[0]

In [ ]:
deck = pd.Series(train_set['Cabin'].values.flatten().astype('str'))
deck1 = []
for i in deck:
    if i != 'nan':
        deck1.append(i[0])
    else: 
        deck1.append(i)

In [ ]:
train_set['Deck']=deck1

In [ ]:
train_set.loc[~(train_set.Cabin.isnull()),['Cabin']].values.flatten()

In [ ]:
train_set['Deck']

In [ ]:
train_set.head(50)

In [ ]:
# Lets see the unique value and count of Deck Column
train_set['Deck'].value_counts()

In [ ]:
train_set.Deck.unique()

In [ ]:
train_set.Deck.isnull().sum()

Lets make the DECK Nan values as GNR. The null values count is same as Cabin

In [ ]:
# Replace Nan in Decek to GNR
train_set['Deck']=train_set['Deck'].replace('nan','GNR')

In [ ]:
train_set['Deck'].value_counts()

In [ ]:
# Remove Cabin Column
train_set.drop('Cabin',axis=1,inplace=True)
test_set.drop('Cabin',axis=1,inplace=True)

In [ ]:
train_set.head()

In [ ]:
# Now lets check the column with null values
train_set.isnull().sum()

Now only 2 records in Embarked have null values. We can either remove them as very less in no or we can replace with value of Max counts of category 

In [ ]:
# Value of Embarked on various categories
train_set.Embarked.value_counts()

In [ ]:
train_set.Embarked.isnull().sum()

In [ ]:
#Lets impute 2 null records of Embarked with value 'S' as it have max occurance
train_set.loc[(train_set.Embarked.isnull()),'Embarked']=train_set.loc[ (train_set.Embarked.isnull()),'Embarked'].replace(np.nan,'S')
train_set.Embarked.isnull().sum()

In [ ]:
# Check if any null columns are present
train_set.isnull().sum()

Now there are no missing values present in the Data. Now Lets do plot some graphs to see distribution of columns and relationship between target variable

In [ ]:
sns.distplot(train_set['Age'])

In [ ]:
g = sns.FacetGrid(train_set, col='Survived',size=5)
g.map(plt.hist, 'Age', bins=30)

**Observations**

* Infants (Age <=4) had high survival rate.
* Oldest passengers (Age = 80) survived.
* Large number of 15-25 year olds did not survive.
* Most passengers are in 15-35 age range.

In [ ]:
# pairplot
sns.pairplot(train_set)
plt.show()

In [ ]:
sns.countplot(x="Pclass", data=train_set)

In [ ]:
# grid = sns.FacetGrid(train_df, col='Pclass', hue='Survived')
grid = sns.FacetGrid(train_set, col='Survived', row='Pclass', size=4, aspect=1.6)
grid.map(plt.hist, 'Age', alpha=.5, bins=30)
grid.add_legend();

**Observations**

* Pclass=3 had most passengers, however most did not survive.
* Infact passengers in Pclass=2 and Pclass=3 mostly survived.
* Most passengers in Pclass=1 survived.
* Pclass varies in terms of Age distribution of passengers.

In [ ]:
sns.distplot(train_set['Fare'])

In [ ]:
sns.boxplot(y=train_set['Fare'])

In [ ]:
# Checking for Outlier
train_set.Fare.describe(percentiles=[.25, .5, .75, .90, .95, .99])

We can see in Fair column we have Outlier, we will check on this using Scalling later

In [ ]:
grid = sns.FacetGrid(train_set, row='Embarked', col='Survived', size=4, aspect=1.6)
grid.map(sns.barplot, 'Sex', 'Fare', alpha=.5, ci=None)
grid.add_legend()

**Observations**

* Higher fare paying passengers had better survival.
* Port of embarkation correlates with survival rates.

In [ ]:
import seaborn as sns
sns.countplot(x="Survived", data=train_set)

In [ ]:
### Checking the Survival Rate Rate
survival = (sum(train_set['Survived'])/len(train_set['Survived'].index))*100
survival

##### 38% of the total have survived which  is good amout of no in target variable.So there is no class imbalance

We can Removed Passanger Id and name as they will not of any use in this case

In [ ]:
# Remove name and Passenger Id Column
train_set.drop(['Name'],axis=1,inplace=True)
test_set.drop(['Name'],axis=1,inplace=True)

In [ ]:
#Checking the Correlation Matrix
plt.figure(figsize = (15,10))
sns.heatmap(train_set.corr(),annot = True)
plt.show()

There is mostly negative correlation between Pclass, Age, Fare rest there is no big Correlation

###### Now lets check the relationship between targert variable and other dependent variable

##### Inferential Stats

In [ ]:
#Check the Survival rate by Paasaenger Class
# print(train_set [['Pclass','Survived']].groupby('Pclass').mean())
a = train_set.groupby(['Pclass','Survived']).agg({'Pclass': 'sum'})
a.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))

Highest Survival in Class 1 and lowest in Class 3

In [ ]:
sns.countplot(x="Pclass", hue="Survived", data=train_set)

Class 3 Passenger are poorly affected with the sink

In [ ]:
sep="---------------------------------------------------------------"
a = train_set.groupby(['Pclass','Sex','Survived']).agg({'Pclass': 'sum'})
a.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
# print(round(a,2),'\n')

In [ ]:
#Check the Survival rate by Paasaenger Class
sep="---------------------------------------------------------------"
print( round(train_set [['Sex','Survived']].groupby(['Sex']).mean()*100,1),'\n',sep)
print(train_set [['Pclass','Sex','Survived']].groupby(['Pclass','Sex']).agg(['count','mean']))

In all the clases the survival rate for Female is higher during the evacuation time Female Passenger were the first to be evcuated we can infer from the above.

96% and 92% females were able to survive on the 1st and 2nd class respectively.

The Survival rate of Male is very less and as 1st is premium class near to the evacuation area o have seen max# of survival in male category. rest keep deminishing

In [ ]:
#tracking the Survival on the basis of Sex and PClass
g = sns.catplot(x="Pclass", hue="Sex", col="Survived",
                data=train_set, kind="count",
                height=4, aspect=.7, size = 7);

In [ ]:
# check the impact of Embarked Colum on Survival
print( round(train_set [['Embarked','Survived']].groupby(['Embarked']).mean()*100,1))

In [ ]:
pd.crosstab(train_set['Survived'],train_set['Pclass']).apply(lambda r: (r/r.sum())*100, axis=1)

In [ ]:
pd.crosstab(train_set['Survived'],[train_set['Pclass'],train_set['Sex']]).apply(lambda r: (r/r.sum())*100, axis=1)

* From the above we can see female has highest survival rate.
* survival rate is also depends on passanger class.

In [ ]:
grid = sns.FacetGrid(train_set, row='Embarked', size=2.2, aspect=1.6)
grid.map(sns.pointplot, 'Pclass', 'Survived', 'Sex', palette='deep')
grid.add_legend()

**Observations**

* Female passengers had much better survival rate than males.
* Exception in Embarked=C where males had higher survival rate. This could be a correlation between Pclass and Embarked and in turn Pclass and Survived, not necessarily direct correlation between Embarked and Survived.
* Males had better survival rate in Pclass=3 when compared with Pclass=2 for C and Q ports.
* Ports of embarkation have varying survival rates for Pclass=3 and among male passengers.

In [ ]:
pd.crosstab(train_set['Survived'],[train_set['Embarked'],train_set['Sex']]).apply(lambda r: (r/r.sum())*100, axis=1)

In [ ]:
train_set.loc[(train_set['Parch']==0)&(train_set['SibSp']==0)]

In [ ]:
sns.countplot(x="Parch", hue="Survived", data=train_set)

In [ ]:
sns.countplot(x="SibSp", hue="Survived", data=train_set)

We can create a new feature for FamilySize which combines Parch and SibSp.

In [ ]:
train_set['Family']=train_set['SibSp']+train_set['Parch']+1
test_set['Family']=test_set['SibSp']+test_set['Parch']+1
train_set.head()

In [ ]:
train_set[['Family', 'Survived']].groupby(['Family'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
train_set['IsAlone'] = 0
test_set['IsAlone'] = 0
train_set.loc[train_set['Family'] == 1, 'IsAlone'] = 1
test_set.loc[test_set['Family'] == 1, 'IsAlone'] = 1
train_set[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [ ]:
df = train_set.groupby(['Ticket']).size().reset_index(name='count')
print(df)

As we can see and multiple people have the same ticket No surprise here as people are travelling with Sibing and Parent. Lets create one more varibale for Ticket Head Count and then we can divide the Total fare per Person

But before the Ticket head Count we need to take test data file for calculating actual no, like how many total passanger were travelling on the same ticket. Then only Fair per person will be the right amount. So for this only combine the test and train data

In [ ]:
# New column for Ticket Head Count on teh complete data
# master=pd.concat([train_set, test_set])
# master.head()
# train_set['TicketHeadCount']=train_set['Ticket'].map(master['Ticket'].value_counts())
train_set['TicketHeadCount']=train_set['Ticket'].map(train_set['Ticket'].value_counts())
test_set['TicketHeadCount']=test_set['Ticket'].map(test_set['Ticket'].value_counts())
train_set.head()

In [ ]:
#Let take fair per Person as per Ticket head Count
train_set['FairPerPerson']=train_set['Fare']/train_set['TicketHeadCount']
test_set['FairPerPerson']=test_set['Fare']/test_set['TicketHeadCount']
train_set[['FairPerPerson']].describe(percentiles=[.25, .5, .75, .90, .95, .99])

There is certainely outlier in Fair and same thing we can see in Fair Per Person

In [ ]:
train_set['FairPerPerson'].value_counts()<1

In [ ]:
# Lets check the distribution
sns.distplot(train_set['FairPerPerson'])

Max no of people have per per ticket cost between 0-$50.0 . As min Fair is zero that is not possible either there is a mistake or it can be the staff those will also be boarded for passanger service. We will check in sometime for this

In [ ]:
#Check the impact of Fair on chances of Survival
plt.figure(figsize = (15,10))
sns.violinplot(x="Family", y="FairPerPerson", hue="Survived",
                    data=train_set, palette="muted")
plt.show()

As % wise the trend is common it is not drawing any impact on Target variable. Less fair or more fair will not impact the cause of Survival realted to family Size kept in between

In [ ]:
sns.violinplot(x="Pclass", y="FairPerPerson", hue="Sex",
                    data=train_set, palette="muted")

As expected the difference in Fair per class

In [ ]:
sns.violinplot(x="Survived", y="Age", hue="Sex",
                    data=train_set, palette="muted")

Few Points:
1. Age group between 0-10 have the good survival% in both gender.
2. Female see the more survives than death in any Age Group
3. After age 60 everybody tends to death.
4. Male sees more death than survivors between 25-35 Age group

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)
sns.stripplot(x="Survived",y="Age",data=train_set.loc[(train_set['Age']>0.0) & (train_set.Age<15.0)],jitter=True,palette='Set1')
plt.subplot(2,3,2)
sns.stripplot(x="Survived",y="Age",data=train_set.loc[(train_set['Age']>15.0) & (train_set.Age<40.0)],jitter=True,palette='Set1')
plt.subplot(2,3,3)
sns.stripplot(x="Survived",y="Age",data=train_set.loc[(train_set['Age']>40.0) & (train_set.Age<60.0)],jitter=True,palette='Set1')
plt.subplot(2,3,4)
sns.stripplot(x="Survived",y="Age",data=train_set.loc[(train_set['Age']>60.0) & (train_set.Age<80.0)],jitter=True,palette='Set1')

plt.show()

Clearly as said earlier the Age Group Section 15-40 have seen more deaths and also major % of travellers

Above 60+ Age the % of death is very high

In [ ]:
# Lets check on graph the survival of male aginst Female within age 15-40 years
sns.stripplot(x="Survived",y="Age",data=train_set.loc[(train_set['Age']>15.0) & (train_set.Age<40.0)],jitter=True,hue='Sex',palette='Set1')

The male Deaths are more dense between 15-40 age group

###### Lets Check on Family Size now

In [ ]:
#tracking th Survival on the basis of Family Size and Sex
g = sns.catplot(x="Family", hue="Sex", col="Survived",
                data=train_set, kind="count",
                height=7, aspect=.7);


1. From the graph the Single Member or solo passanger survival rate is very poor.
2. Male Solo Passanger number is quite high and mortality rate too in comparision too Female.
3. Survival Chances become low after family size>4

Let us drop Parch, SibSp, and FamilySize features in favor of IsAlone.

In [ ]:
sns.boxplot(x = 'Pclass', y = 'FairPerPerson',hue='Survived', data = train_set)

On Fare Per Person among Survival and non survivers are same . Only 1st class have some outliers

In [ ]:
# Group the Deck by Class
print(train_set.groupby([ 'Pclass','Deck'])['Survived'].agg(['count','mean']))

In [ ]:
# Lets Check the pattern of Deck on Age
sns.swarmplot(x="Deck",y="Age",hue='Sex',data=train_set,palette="Set1", split=True)

The Deck are randomly distributed over. GNR which we named as unknown have highest distribution.

2 reasons->Either Deck no on others are missing or Deck space were small rooms

In [ ]:
sns.swarmplot(x="Deck",y="FairPerPerson",hue='Pclass',data=train_set,palette="Set1", split=True)

Even over the class the Deck are present for each. So nothing surprises here.

In [ ]:
plt.figure(figsize=(25, 14))
sns.catplot(x="Deck", col="Survived",data=train_set, kind="count",height=4, aspect=.7, hue='Pclass')
plt.show()

cabin column is not providing any insights majority GNR class has less survival rate which we imputed.

In [ ]:
print(train_set.loc[(train_set['FairPerPerson']==0),['Embarked','Ticket','SibSp','Parch','Age','Sex','Family','TicketHeadCount']])

1. From this above data it looks like these are Line man and Crew member. 
2. All have the same boarding station and Ticket no on some says"Line". 
3. All are male and of approx average age group 25-45 years.
4. These are solo  travellers no sibiling or parent

In [ ]:
train_set['FairPerPerson'].describe(percentiles=[.25, .5, .75, .90, .95, .99])

In [ ]:
train_set.info()

In [ ]:
quantile_1, quantile_3 = np.percentile(train_set.FairPerPerson, [25, 75])

In [ ]:
print(quantile_1, quantile_3)

In [ ]:
iqr_value = quantile_3 - quantile_1
iqr_value

In [ ]:
lower_bound_val = quantile_1 - (1.5 * iqr_value)
upper_bound_val = quantile_3 + (1.5 * iqr_value)
print(lower_bound_val, upper_bound_val)

In [ ]:
plt.figure(figsize = (10, 5))
sns.kdeplot(train_set.FairPerPerson)
plt.axvline(x=lower_bound_val, color = 'red')
plt.axvline(x=upper_bound_val, color = 'red')

In [ ]:
train_set[(train_set.FairPerPerson >= lower_bound_val) & (train_set.FairPerPerson <= upper_bound_val)].info()

In [ ]:
round(100*(train_set[(train_set.FairPerPerson >= lower_bound_val) & (train_set.FairPerPerson <= upper_bound_val)].count()/len(train_set.index)), 2)

In [ ]:
round(100*(train_set[(train_set.FairPerPerson >= 0) & (train_set.FairPerPerson <= 100)].count()/len(train_set.index)), 2)

We will choose some Fare range which we set max to leave out after that.
1. People with Fare 0  we will exclude them.
2. People with fair greater than 60 we will exclude them

By doing this we will only loose approve 1% of data and that is on which we can still live with it.

In [ ]:
train_set_copy=train_set.loc[(train_set.FairPerPerson>0) & (train_set.FairPerPerson<=100)]
train_set_copy.shape

In [ ]:
train_set_copy.info()

In [ ]:
sns.boxplot(x = 'Pclass', y = 'FairPerPerson',hue='Survived', data = train_set_copy)

#### EDA Discovery
1. Female Gender is highly survived on the ship regardless of any class against male.
2. People with 1st Class have low casulity rate.
3. Pclass and Fair are co-linear and will explain the same impact on Targer variable. As Fair increase, class increase and with that your survival chances do increase.
4. Travelling in a group of 2-4 will increase your chances of getting saved.
5. Senior Citizen on lower class will not be able to make it out from the casulity.
6. 25-40 age group will have more chances to get saved.

##### Dropping some columns which we will impact on Survival.
1. Title is used for imputing Age missing values and things can be drawn out of Age column.
2. Will drop Parch, Sibsp as we derived Family and IsAlone column.
3. Will drop Ticket, Deck, Ticket Head Count Column as it won't give any impact.
4. Will drop Fare columns as we derived FairPerPerson column to use

In [ ]:
train_set.head()

In [ ]:
test_set.head()

In [ ]:
train_set_copy.drop(['Parch','Ticket','Fare','Deck','SibSp','TicketHeadCount'],axis=1,inplace=True)
test_set.drop(['Parch','Ticket','Fare','SibSp','TicketHeadCount'],axis=1,inplace=True)
train_set_copy.head()

##### For categorical variables with multiple levels, create dummy features (one-hot encoded)


In [ ]:
train_set_copy = pd.concat([train_set_copy, pd.get_dummies(train_set_copy['Sex'], drop_first=True)], axis=1)
test_set = pd.concat([test_set, pd.get_dummies(test_set['Sex'], drop_first=True)], axis=1)

In [ ]:
train_set_copy['Embarked'] = train_set_copy['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
test_set['Embarked'] = test_set['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [ ]:
#Drop Original Columns
train_set_copy.drop(['Sex'],axis=1,inplace=True)
test_set.drop(['Sex'],axis=1,inplace=True)

##### Test-Train Split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Feature Scaling

In [ ]:
X_train = train_set_copy.drop("Survived", axis=1)
y_train = train_set_copy["Survived"]
X_test  = test_set.drop("PassengerId", axis=1).copy()
X_train.shape, y_train.shape, X_test.shape

In [ ]:
scaler = StandardScaler()

X_train[['Age','FairPerPerson']] = scaler.fit_transform(X_train[['Age','FairPerPerson']])
X_test[['Age','FairPerPerson']] = scaler.fit_transform(X_test[['Age','FairPerPerson']])

X_train.head()

In [ ]:
### Checking the Survival Rate
Survival = (sum(train_set_copy['Survived'])/len(train_set_copy['Survived'].index))*100
Survival

In [ ]:
plt.figure(figsize = (20,10))
sns.heatmap(X_train.corr(),annot = True)
plt.show()

## Model Building

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
# Importing classification report and confusion matrix from sklearn metrics
from sklearn.metrics import classification_report,accuracy_score

###### Defualt Hyperparameters
Let's first fit a random forest model with default hyperparameters.

In [ ]:
col = [ 'Pclass', 'Age', 'Embarked', 'Title', 'Family',
       'IsAlone', 'FairPerPerson', 'male']

In [ ]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train[col], y_train)
Y_pred = logreg.predict(X_test[col])
acc_log = round(logreg.score(X_train[col], y_train) * 100, 2)
acc_log

In [ ]:
# Support Vector Machines

svc = SVC()
svc.fit(X_train[col], y_train)
Y_pred = svc.predict(X_test[col])
acc_svc = round(svc.score(X_train[col], y_train) * 100, 2)
acc_svc

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train[col], y_train)
Y_pred = knn.predict(X_test[col])
acc_knn = round(knn.score(X_train[col], y_train) * 100, 2)
acc_knn

In [ ]:
gaussian = GaussianNB()
gaussian.fit(X_train[col], y_train)
Y_pred = gaussian.predict(X_test[col])
acc_gaussian = round(gaussian.score(X_train[col], y_train) * 100, 2)
acc_gaussian

In [ ]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train[col], y_train)
Y_pred = perceptron.predict(X_test[col])
acc_perceptron = round(perceptron.score(X_train[col], y_train) * 100, 2)
acc_perceptron

In [ ]:
# Linear SVC

linear_svc = LinearSVC()
linear_svc.fit(X_train[col], y_train)
Y_pred = linear_svc.predict(X_test[col])
acc_linear_svc = round(linear_svc.score(X_train[col], y_train) * 100, 2)
acc_linear_svc

In [ ]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train[col], y_train)
Y_pred = sgd.predict(X_test[col])
acc_sgd = round(sgd.score(X_train[col], y_train) * 100, 2)
acc_sgd

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train[col], y_train)
Y_pred = decision_tree.predict(X_test[col])
acc_decision_tree = round(decision_tree.score(X_train[col], y_train) * 100, 2)
acc_decision_tree

In [ ]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train[col], y_train)
Y_pred = random_forest.predict(X_test[col])
random_forest.score(X_train[col], y_train)
acc_random_forest = round(random_forest.score(X_train[col], y_train) * 100, 2)
acc_random_forest

In [ ]:
models = pd.DataFrame({
    'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
              'Random Forest', 'Naive Bayes', 'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
              'Decision Tree'],
    'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree]})
models.sort_values(by='Score', ascending=False)

From the Above represtation we can clearly see the Random forest and Decision Model Performace is better among all.

In [ ]:
# Random Forest
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train[col], y_train)
Y_pred = random_forest.predict(X_test[col])
random_forest.score(X_train[col], y_train)
acc_random_forest = round(random_forest.score(X_train[col], y_train) * 100, 2)
acc_random_forest

# Hyperparameter tunning

In [ ]:
# Cross validate model with Kfold stratified cross val
kfold = StratifiedKFold(n_splits=10)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve
# Adaboost
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsadaDTC.fit(X_train[col],y_train)

ada_best = gsadaDTC.best_estimator_
gsadaDTC.best_score_

In [ ]:
#ExtraTrees 
ExtC = ExtraTreesClassifier()


## Search grid for optimal parameters
ex_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsExtC = GridSearchCV(ExtC,param_grid = ex_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsExtC.fit(X_train[col],y_train)

ExtC_best = gsExtC.best_estimator_

# Best score
gsExtC.best_score_

In [ ]:
# RFC Parameters tunning 
RFC = RandomForestClassifier()


## Search grid for optimal parameters
rf_param_grid = {"max_depth": [None],
              "max_features": [1, 3, 10],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [False],
              "n_estimators" :[100,300],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(RFC,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_

In [ ]:
# Gradient boosting tunning

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train[col],y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

In [ ]:
### SVC classifier
SVMC = SVC(probability=True)
svc_param_grid = {'kernel': ['rbf'], 
                  'gamma': [ 0.001, 0.01, 0.1, 1],
                  'C': [1, 10, 50, 100,200,300, 1000]}

gsSVMC = GridSearchCV(SVMC,param_grid = svc_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsSVMC.fit(X_train[col],y_train)

SVMC_best = gsSVMC.best_estimator_

# Best score
gsSVMC.best_score_

In [ ]:
votingC = VotingClassifier(estimators=[('rfc', RFC_best), ('extc', ExtC_best),
('svc', SVMC_best), ('adac',ada_best),('gbc',GBC_best)], voting='soft', n_jobs=4)

votingC = votingC.fit(X_train[col], y_train)
Y_pred = votingC.predict(X_test[col])

In [ ]:
from xgboost.sklearn import XGBClassifier
model = XGBClassifier(learning_rate=0.001,n_estimators=2500,
                                max_depth=4, min_child_weight=0,
                                gamma=0, subsample=0.7,
                                colsample_bytree=0.7,
                                scale_pos_weight=1, seed=27,
                                reg_alpha=0.00006)
model.fit(X_train[col], y_train)
Y_pred = model.predict(X_test[col])

In [ ]:
submission = pd.DataFrame({
        "PassengerId": test_set["PassengerId"],
        "Survived": Y_pred
     })
submission.to_csv('/kaggle/working/submission.csv', index=False)